In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from numpy import exp
from scipy import ndimage
from VMOSfunc import *
from astropy.table import Table, hstack
import os
import glob
from astropy.table import Table, vstack, join
import pandas as pd
from shutil import copyfile

In [2]:
def err_ratio(data,err):
    data[2:-2,2:-2]=0
    err[2:-2,2:-2]=0
    edge_fluxmap = data[data!=0]
    edge_errmap = err[err!=0]
    err_fluxmap = np.std(edge_fluxmap)
    err_errmap = np.mean(edge_errmap)
    fact = err_fluxmap/err_errmap
    return fact

def emp_fact(err_ratio_Hb1,err_ratio_Hb2,err_ratio_OIII_br,broad2=False):
    if broad2:
        emp_Hb2 = err_ratio_Hb2
    else:
        emp_Hb2 = 0
    emp_Hb1 = err_ratio_Hb1
    emp_wing = err_ratio_OIII_br
    return emp_Hb1,emp_Hb2,emp_wing

In [4]:
def emp_table(obj,emp_Hb1,emp_Hb2,emp_wing,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/VIMOS"):
    popt = [emp_Hb1,emp_Hb2,emp_wing]
    column_names={'emp_fact_Hb1':0,'emp_fact_Hb2':1,'emp_fact_wing':2}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key,format='E',array=[popt[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hdu = fits.BinTableHDU.from_columns(coldefs)
    hdu.writeto('%s/%s/%s_19_arcsec_empirical_factor_subcube.fits'%(destination_path_cube,obj,obj),overwrite=True)

def emp_fact_table(obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/VIMOS"):
    t1 = Table.read('%s/%s/source_%s.fits'%(destination_path_cube,obj,obj),format='fits')
    t2 = Table.read('%s/%s/%s_19_arcsec_empirical_factor_subcube.fits'%(destination_path_cube,obj,obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s/%s/%s_19_arcsec_empirical_factor_subcube.fits'%(destination_path_cube,obj,obj),overwrite=True)

In [5]:
def algorithm_script(obj,z,broad2,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/VIMOS"):
    print ('%s'%(obj))

    (Hb1_blr_br_data,Hb2_blr_br_data) = loadblr('%s/Flux Maps/%s/19_arcsec_subcube_par_%s.fits'%(destination_path_cube,obj,obj))
    (Hb1_blr_br_err,Hb2_blr_br_err) = loadblr('%s/Flux Maps/%s/19_arcsec_subcube_par_err_%s.fits'%(destination_path_cube,obj,obj))
    OIII_br_err = loadwing_err('%s/Flux Maps/%s/19_arcsec_subcube_par_err_%s.fits'%(destination_path_cube,obj,obj))
    OIII_br_data = loadwing_err('%s/Flux Maps/%s/19_arcsec_subcube_par_%s.fits'%(destination_path_cube,obj,obj)) # It opens the hdu[3] where the OIII_br data 
  
    err_ratio_OIII_br = err_ratio(OIII_br_data,OIII_br_err)
    err_ratio_Hb1 = err_ratio(Hb1_blr_br_data,Hb1_blr_br_err)
    err_ratio_Hb2 = err_ratio(Hb2_blr_br_data,Hb2_blr_br_err)
    (emp_Hb1,emp_Hb2,emp_wing) = emp_fact(err_ratio_Hb1,err_ratio_Hb2,err_ratio_OIII_br,broad2)
    print (emp_Hb1,emp_Hb2,emp_wing)
    emp_table(obj,emp_Hb1,emp_Hb2,emp_wing)
    #emp_fact_table(obj)

In [6]:
z = {"HE1310-1051":0.03427,"HE1338-1423":0.04179}

broad2= {"HE1310-1051":False,"HE1338-1423":True}

objs = z.keys()

for obj in objs:
    algorithm_script(obj,z[obj],broad2[obj])

HE1310-1051
2.522232 0 2.3234367
HE1338-1423
2.7409718 3.9046783 4.7387133


For this galaxy when we did not have the proper error propagation estimating code 
and continuum missing from the broad H-beta the reduced chi-squared was 23. Introducing 
the continuum brought it down to 14. Then we did the error estimation with monte carlo 
properly which further brought it down to 7. Now we see that the error for the both 
BLR H-beta and broad [OIII] was underestimated. 

What we did here is to check the error from the 2D maps and the 2D error maps to see whether
they make any sense. That is why we choose the region of the edge where we took 2 pixels from 
the edges along the map and calculate the standard deviation. The reason behind this is BLR
H-beta is point like and hence along the edge it should be almost equal to 0 but from the 2D
maps it is not cleraly 0 but a fluctuation between positive and sometimes negative values.
Taking the standard deviation would give us the error.

Next we take the 2D error maps and choose the corresponding pixels. Since this is an error map
already, thi stime we calculate the mean. Taking the reatios of this two error we get a ratio of
5 which means that the true error was underestimated to a factor of 5. This factor is constant 
throughout the edge.

We also do it for the broad [OIII] since most of the [OIII] emission comes from a very compact
region around the center. Hence, if there is any excess flux that should be rather low and will
not interfere with our measurements. Repeating the previous steps we get a ratio of 2.5.

We cannot do it for the narrow [OIII] since it is extended in nature and hence the error computation
from the 2D maps will not return its true eeror.